## 조달청 예산 사업설명자료 바탕, 평생교육 등의 법령 기반 텍스트 추출

<oi>
    조달청 예산 사업 설명자료에서, 법령 단어 일부인 "교육, 훈련", "자기계발", "자기개발", "인재양성" 등의 단어가 포함된 <br>
    사업을 추출하여, 형식에 맞추어, 법적근거, 사업의 위계, 예산 등등을 데이터 전처리 하는 과정을 진행해봄 <br>
    법령.csv와 같은 경우는 필요한 법령을 기반으로 법적 근거가 일치하는 사업의 데이터를 전처리 하는 과정에 포함되어 있으나 <br>
    타 코드에서는 구현을 해서 완료하였으나, 정보 등의 문제로 github에는 기재하지 않음, 방식은 유사함 

In [14]:
import pandas as pd
law = pd.read_csv("법령단어.csv")
law = law[law['B1_1'] =='조달청']
law.head()

,B1_1,B3,B4
42,조달청,교육,NaN
43,조달청,훈련,NaN
44,조달청,자기계발,NaN
45,조달청,자기개발,NaN
46,조달청,인재양성,NaN


In [13]:
import random
import string

law1 = pd.read_csv("법령.csv")
law1 = law1[law1['B1_1'] =='조달청']
law1.columns = range(1, 39) #내부 정보 상 컬럼명 데이터 명 변경함
law1.head()

,1,2,3,4,5,6,7,8,9,10,...,29,30,31,32,33,34,35,36,37,38
193,229.0,229.0,195.0,이슈페이퍼 분석,제17816호,2021. 1. 5.,2021. 7. 6.,조달청,규제개혁법무담당관실,제19편 재정/경제일반,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
789,907.0,907.0,810.0,이슈페이퍼 분석,제17354호,2020. 6. 9.,2020. 12. 10.,조달청,전자조달기획과,제19편 재정/경제일반,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
811,930.0,930.0,832.0,이슈페이퍼 분석,제17348호,2020. 6. 9.,2020. 12. 10.,조달청,규제개혁법무담당관실,제19편 재정/경제일반,...,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [1]:
#해양수산부, 통계청 둘 모두 증감 -2가 위치함
import pandas as pd
import re
import numpy as np

nn = ["조달청"]
res = pd.read_csv("견본.csv")
m=0

for n in range (0, len(nn)):
    law = pd.read_csv("법령단어.csv")
    law=law[["B1_1", "B3", "B4"]]
    law["B3"] = law["B3"].str.replace("\n", '')
    san = law[law["B1_1"]==nn[n]].reset_index(drop=True)
    for i in range(0, len(san)):
        law1 = san.loc[i, "B3"].strip()
        
        law2 = san.loc[i, "B4"]
        if not pd.isna(san.loc[i, "B4"]):
            law2=law2.strip()
        else:
            law2=""



        a=pd.read_csv(nn[n]+".csv", encoding="cp949", low_memory=False)
        df =a.dropna(how="all")
        df.reset_index(drop=True, inplace=True)
        # 정규 표현식 패턴 생성: law1 또는 law2가 포함된 경우
        pattern = re.compile(fr"(?=.*{re.escape(law1)})(?=.*{re.escape(law2)})")
        target_indexes = df[df.iloc[:, 0].str.contains(pattern).replace(np.nan, False)].index
        start = 0
        if(len(target_indexes)>0):
            for target_index in target_indexes:
                print(df.iloc[target_index, 0])
                for index in range(target_index - 1, -1, -1):  # 특정 인덱스 이전 행을 역순으로 확인
                    if df.iloc[index, 0] == "사   업   명":
                        사업명_바로_밑_값 = df.iloc[index + 1, 0]
                        #print(사업명_바로_밑_값)
                        res.loc[m, '기관명'] = nn[n]
                        res.loc[m, '사업명'] = 사업명_바로_밑_값
                        start = index
                        법적근거 = san.loc[i, "B3"] + " "+ law2
                        res.loc[m, '법적근거'] = 법적근거
                        위계 = df.iloc[start + 6, :].dropna()[-1]
                        res.loc[m, '사업의 위계'] = 위계
                        #print(위계)
                        for index, value in enumerate(df.iloc[start + 3, :]):
                            if value == "분야":
                                column_index = index
                                break
                        분야 = df.iloc[start + 5, column_index]
                        res.loc[m, '분야'] = 분야

                        for index, value in enumerate(df.iloc[start + 3, :]):
                            if value == "부문":
                                column_index = index
                                break
                        부문 = df.iloc[start + 5, column_index]
                        res.loc[m, '부문'] = 부문

                        for index, value in enumerate(df.iloc[start + 3, :]):
                            if value == "회계" or value == "기금":
                                column_index = index
                                break
                        회계 = df.iloc[start + 4, column_index]
                        res.loc[m, '출처'] = 회계
            #            print(분야)
            #            print(부문)
            #            print(회계)


                        for index1 in range(target_index - 1, -1, -1):  # 특정 인덱스 이전 행을 역순으로 확인
                            if not pd.isna(df.iloc[index1, 0]):
                                if "예산 총괄표" in df.iloc[index1, 0] or "예산안 총괄표" in df.iloc[index1, 0] or "지출계획 총괄표" in df.iloc[index1, 0]  or "지출계획안 총괄표" in df.iloc[index1, 0]: # 가. 가 아닌경우 존재df.iloc[index1, 0] == "가. 예산 총괄표":
                                    for index, value in enumerate(df.iloc[index1 + 3, :]):
                                        if value == "증감 (B-A)" or value == "증감":
                                            column_index = index
                                            예산확정값 = df.iloc[index1 + 5, column_index-2]
                                            res.loc[m, '금액(백만원)'] = 예산확정값
                                            break
                                    #print(예산확정값)
                                    start = index1
                                    break
                        else:
                            print("예산의 해당하는 데이터가 없습니다.")


                        for index in range(target_index + 1,  min(target_index +200,len(df))):  # target_index 이후의 행을 순차적으로 확인
                            if not pd.isna(df.iloc[index, 0]):
                                if "사업 수혜자" in df.iloc[index, 0]:
                                    res.loc[m, '수혜대상'] = df.iloc[index, 0]
                                    #print(df.iloc[index, 0])
                                    break
                        break
                else:
                    print("사업명의 해당하는 데이터가 없습니다.")

                m +=1
            사업명_바로_밑_값 = np.nan
            법적근거 = np.nan
            위계 = np.nan
            분야 = np.nan
            부문 = np.nan
            회계 = np.nan
            예산확정값 = np.nan
            
            
res.drop_duplicates(inplace=True)
res.to_csv('res_2_단어_조달청.csv', index=False, encoding="CP949")

  ⑮ (7134-371) 조달전문교육 운영	74
사업명의 해당하는 데이터가 없습니다.
  &#9327t (7134-650) 조달전문교육 수입대체경비	78
사업명의 해당하는 데이터가 없습니다.
&#9702t 공무원 인재개발법 제16조(교육훈련기관의 운영) ① 공무원교육훈련기관의 장은 해당 교육훈련기관을 효율적으로 운영하기 위하여 필요하면 교육훈련시설 및 교육훈련과정 등을 국가기관, 공공단체 또는 민간에 유상(有償)으로 제공할 수 있다.
사업명의 해당하는 데이터가 없습니다.
&#9702t 조달사업에 관한 법률 제7조(조달교육) 조달청장은 조달청, 수요기관 및 민간업체의 조달업무나 납품업무 종사자의 전문성과 자질 향상을 위한 조달교육을 할 수 있다. 
사업명의 해당하는 데이터가 없습니다.
&#9702t 조달전문교육 과정에 참여하는 교육생이 납부하는 교육비와 기숙사비 
사업명의 해당하는 데이터가 없습니다.
  - (내역사업 : 전산장비 운영유지) 나라장터와 조달정보시스템의 효율적·안정적 운영을 위한 네트워크 회선비 등 공공요금, 정보화교육, 전산소모품 구매, 노후장비 교체(임차료), 국내 여비 등 전산장비를 운영 유지하기 위한 사업비
제4조의4(해외 조달시장 진출에 대한 지원) ① 조달청장은 법 제3조의6제1항에 따라 다음 각 호의 방법으로 국내기업의 해외 조달시장 진출을 지원할 수 있다. 1. 해외 조달시장 및 공공 입찰에 관한 정보 제공 2. 국내기업의 해외 조달시장 진출 역량을 강화하기 위한 교육훈련 3. 조달 물품ㆍ용역관련 기술 및 인력의 국제교류 등에 필요한 사업의 지원 4. 그 밖에 국내 기업의 해외 조달시장 진출에 필요하다고 인정되는 사항의 지원 ② 조달청장은 제1항에 따른 지원 업무를 수행하는 데 필요한 경비의 전부 또는 일부를 예산의 범위에서 지원할 수 있다.
     - G-PASS기업 대상으로 해외마케팅(해외전시회 및 시장개척단 파견, 유엔 조달시장 진출), 시장 정보제공, 수출전략기업 육성, 실무교육 등 지원 수행 
(15) 조달전문교육 운영 

In [2]:
import pandas as pd
res=pd.read_csv('res_2_단어_조달청.csv', encoding="CP949")
res

,기관명,사업명,법적근거,사업의 위계,분야,부문,출처,금액(백만원),수혜대상
0,조달청,(1) 조달정보화(정보화) (1131 &#8211t 501),교육,세부사업,일반·지방행정,일반행정,조달특별회계,"27,818","ㅇ 사업 수혜자 : 공공기관, 조달업체 및 국민"
1,조달청,(5) 정부조달 국제협력체제 구축(1233 &#8211t 331),교육,세부사업,일반공공행정,일반행정,조달특별회계,"3,141","ㅇ 사업 수혜자 : 국가, 조달관련 기업 등"
2,조달청,(15) 조달전문교육 운영 (7134 - 371),교육,세부사업,일반공공행정,일반행정,조달특별회계,"5,945",ㅇ 사업 수혜자 : 조달청과 수요기관 및 민간업체의 조달업무 종사자
3,조달청,(15) 조달전문교육 운영 (7134 - 371),교육,세부사업,일반공공행정,일반행정,조달특별회계,"1,211",ㅇ 사업 수혜자 : 조달청과 수요기관 및 민간업체의 조달업무 종사자
4,조달청,(16) 조달전문교육 수입대체경비 (7134 - 650),교육,세부사업,일반공공행정,일반행정,조달특별회계,"1,211",ㅇ 사업 수혜자 : 조달청과 수요기관 및 민간업체의 조달업무 종사자
5,조달청,(16) 조달전문교육 수입대체경비 (7134 - 650),교육,세부사업,일반공공행정,일반행정,조달특별회계,267,ㅇ 사업 수혜자 : 조달청과 수요기관 및 민간업체의 조달업무 종사자
6,조달청,(5) 정부조달 국제협력체제 구축(1233 &#8211t 331),훈련,세부사업,일반공공행정,일반행정,조달특별회계,"3,141","ㅇ 사업 수혜자 : 국가, 조달관련 기업 등"
7,조달청,(15) 조달전문교육 운영 (7134 - 371),훈련,세부사업,일반공공행정,일반행정,조달특별회계,"1,211",ㅇ 사업 수혜자 : 조달청과 수요기관 및 민간업체의 조달업무 종사자
8,조달청,(16) 조달전문교육 수입대체경비 (7134 - 650),훈련,세부사업,일반공공행정,일반행정,조달특별회계,267,ㅇ 사업 수혜자 : 조달청과 수요기관 및 민간업체의 조달업무 종사자
9,조달청,(15) 조달전문교육 운영 (7134 - 371),자기개발,세부사업,일반공공행정,일반행정,조달특별회계,"1,211",ㅇ 사업 수혜자 : 조달청과 수요기관 및 민간업체의 조달업무 종사자
